In [5]:
import base64
import requests
from bs4 import BeautifulSoup as bs
from .secrets import Secret 
import json

class MakeHtml:
    def __init__(self):
        secret = Secret()
        # confluence OAuth AppId and Secrets
        self.confluenceAppId = secret.confluenceAppId
        self.confluenceSecretkey = secret.confluenceSecretkey
        
        # redirectUrl
        self.redirectUrl = 'https://oauth.modutech.win/oauth/confluence'
        
        # imgBB api key
        self.imgApiKey = secret.apiKeyImgbb
        # user's confluence space baseurl
        self.baseUrl = "https://api.atlassian.com/ex/confluence"
        # header for api requests
        self.headers = {"Authorization": "Bearer {token}", 'Accept': 'application/json'}
        # htmlBodySoup
        self.htmlSoup = ""
    
    def generate_random_key(length):
        return ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(length))
    
    def oauthMakeLink(self):
        import random
        import string

        state = self.generate_random_key(30)
        makeLinkUrl = 'https://auth.atlassian.com/authorize'
        params = {
            'audience': 'api.atlassian.com',
            'client_id': self.confluenceAppId,
            'scope': 'read:confluence-space.summary read:confluence-content.all read:confluence-content.summary read:confluence-props search:confluence',
            'redirect_uri': self.redirectUrl,
            'state': state,
            'response_type': 'code',
            'prompt': 'consent'
        }
        makedUrl = requests.Request('GET', makeLinkUrl, params=params).prepare().url
        
        return makedUrl, state
    
    # fill api access information and check api can speak
    def loginProcess(self, authCode):
        getTokenUrl = 'https://auth.atlassian.com/oauth/token'
        headers = self.headers
        headers['Content-Type'] = 'application/json'
        jsonData = {
            'grant_type': 'authorization_code',
            'client_id': self.confluenceAppId,
            'client_secret': self.confluenceSecretkey,
            'code': authCode,
            'redirect_uri': self.redirectUrl
        }
        authRes = requests.post(getTokenUrl, json=jsonData, headers=headers)
        print (authRes.text)
        if (authRes.status_code == 200):
            token = authRes.json()['access_token']
            
            self.headers["Authorization"] = "Bearer {}".format(token)
            print ("Bearer {}".format(token))
            print ('Success to get Token!')
            return "Bearer {}".format(token)
        else:
            print ('Auth Fail!')
            return False
        

        
        # check api
#         testUrl = "{}/rest/api/space".format(self.baseUrl)
#         print (testUrl)
#         testRes = requests.get(getBaseUrl, headers=self.headers)
#         if (testRes.status_code == 200):
#             print ('Success login!')
#         else:
#             print (testRes.status_code)
#             return False
        
    # get domain
    def getDomain(self):
        getBaseUrl = 'https://api.atlassian.com/oauth/token/accessible-resources'
        getBaseRes = requests.get(getBaseUrl, headers=self.headers)
        if (getBaseRes.status_code == 200):
            print (getBaseRes.json())
#             baseId = getBaseRes.json()[1]['id']
#             self.baseUrl = 'https://api.atlassian.com/ex/confluence/{}'.format(baseId)
            print ("Success get Id of domain!")
            return (getBaseRes.json())
        else:
            print (getBaseRes.status_code)
            return False
        
        
    
    # get space list from 
    def getSpaceList(self, baseId):
        spaceListUrl = "{}/{}/rest/api/space".format(self.baseUrl, baseId)
        res = requests.get(spaceListUrl, headers=self.headers).json()
        
        # spaceListDict {"sample": {"id":1234, "key": "SPACE1", "selfUrl": "https://example.com"}}
        spaces = {}
        for item in res['results']:
            spaces[item['name']] = {}
            spaces[item['name']]['id'] = item['id']
            spaces[item['name']]['key'] = item['key']
        print (spaces)
        return spaces

    def getContentList(self, baseId, spaceKey):
        params = {"spaceKey": spaceKey}
        contentListUrl = "{}/{}/rest/api/content".format(self.baseUrl, baseId)
        res = requests.get(contentListUrl, params=params, headers=self.headers).json()
        
        # contentListDict {"id(num)": "title"}
        contents = {}
        for item in res['results']:
            contents[item['id']] = item['title']
            
        print (contents)
        return contents
    
    def getCententHtml(self, baseId, contentId):
        viewType = "view"
        params = {"expand": "body.{}".format(viewType)}
        contentUrl = "{}/{}/rest/api/content/{}".format(self.baseUrl, baseId, contentId)
        res = requests.get(contentUrl, params=params, headers=self.headers).json()
        htmlBody = res['body'][viewType]['value']
        self.htmlSoup = bs(htmlBody, 'html.parser')
        self.rebuildFormat()
        self.rebuildImgStore()
        return self.htmlSoup
        
        
    def rebuildFormat(self):
        # wrap with single tag
        new_tag = self.htmlSoup.new_tag('content')
        body_children = list(self.htmlSoup.children)
        self.htmlSoup.clear()
        self.htmlSoup.append(new_tag)
        for child in body_children:
            new_tag.append(child)  
        
        # replace all 'th' tag to 'tr' tag
        for th in self.htmlSoup.findAll('th'):
            th.name = 'td'
        
        # beautify table view
        for table in self.htmlSoup.findAll('table'):
            table.attrs['data-ke-style'] = 'style12'
           
        # append <code> tag inside <pre> tag for beautify code block
        for pre in self.htmlSoup.findAll('pre', class_='syntaxhighlighter-pre'):
            newTag = self.htmlSoup.new_tag('code')
            newTag.string = pre.string
            pre.string = ""
            pre.append(newTag)

            
    # upload image to imgbb and return image's url
    def uploadImg(self, imgBin):
        imgUpload = "https://api.imgbb.com/1/upload"
        params = {'key': self.imgApiKey}
        imgBase64 = base64.b64encode(imgBin).decode('ascii')
        data = {'image': imgBase64}
        res = requests.post(imgUpload, params=params, data=data).json()
        return res['data']['url']
        
    # replace <img> tag for public viwer
    def rebuildImgStore(self):
        for img in self.htmlSoup.findAll('img'):
            imgSrc = img.attrs['src']
            res = requests.get(imgSrc, headers=self.headers)
            
            imgUrl = self.uploadImg(res.content)
            # replace <img> tag's src and delete not use attrs
            img.attrs['src'] = imgUrl
            img.attrs['data-image-src'] = ""
            img.attrs['data-base-url'] = ""
        print (self.htmlSoup)
    
    def saveHtmlFile(self, fileName):
        with open("{}.html".format(fileName), "w", -1, "utf-8") as f:
            f.write(str(self.htmlSoup))

            
def main():
    make1 = MakeHtml()
    link = make1.oauthMakeLink()
    print (link)
    authCode = input('Auth Code : ')
    make1.loginProcess(authCode)
    
    make1.getDomain()
    
    baseId = input ('baseId : ')
    make1.getSpaceList(baseId)

    spaceName = input ('input space key : ')
    lists = make1.getContentList(baseId, spaceName)

    contendId = input('input content ID : ')
    htmlSoup = make1.getCententHtml(baseId, contendId)
    fileName = input('파일명을 입력하세요 : ')
    make1.saveHtmlFile('output1.html')
    return htmlSoup

if __name__ == "__main__":

    main()

https://auth.atlassian.com/authorize?audience=api.atlassian.com&client_id=CnxeVmMdZBx2ya47MBR8a0czQmOkwm9J&scope=read%3Aconfluence-space.summary+read%3Aconfluence-content.all+read%3Aconfluence-content.summary+read%3Aconfluence-props+search%3Aconfluence&redirect_uri=https%3A%2F%2Foauth.modutech.win%2Foauth%2Fconfluence&state=ne964mmenjz3vc5xvj7sl7bgdnkslm&response_type=code&prompt=consent
Auth Code : NeULD1MKFzrORpYf
{"access_token":"eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6Ik16bERNemsxTVRoRlFVRTJRa0ZGT0VGRk9URkJOREJDTVRRek5EZzJSRVpDT1VKRFJrVXdNZyJ9.eyJodHRwczovL2F0bGFzc2lhbi5jb20vb2F1dGhDbGllbnRJZCI6IkNueGVWbU1kWkJ4MnlhNDdNQlI4YTBjelFtT2t3bTlKIiwiaHR0cHM6Ly9hdGxhc3NpYW4uY29tL2VtYWlsRG9tYWluIjoiZ21haWwuY29tIiwiaHR0cHM6Ly9hdGxhc3NpYW4uY29tL3N5c3RlbUFjY291bnRJZCI6IjYwNjFkN2FmOTBlMzk1MDA2OWE3NDE4NiIsImh0dHBzOi8vYXRsYXNzaWFuLmNvbS9zeXN0ZW1BY2NvdW50RW1haWxEb21haW4iOiJjb25uZWN0LmF0bGFzc2lhbi5jb20iLCJodHRwczovL2F0bGFzc2lhbi5jb20vdmVyaWZpZWQiOnRydWUsImh0dHBzOi8vYXRsYXNzaWFuLmNvbS9maXJzdFBhcn

KeyboardInterrupt: Interrupted by user

In [ ]:
make1 = MakeHtml()
link = make1.oauthMakeLink()
print (link)
authCode = input('Auth Code : ')
userName = "geunsam2"
make1.loginProcess(authCode, userName)
make1.getSpaceList()

spaceName = input ('input space key : ')
lists = make1.getContentList(spaceName)

contendId = input('input content ID : ')
htmlSoup = make1.getCententHtml(contendId)

In [3]:
MakeHtml

TypeError: oauthMakeLink() missing 1 required positional argument: 'self'

In [8]:
import requests

while (True):
    res = requests.get('http://192.168.0.249:10217/1.mkv')

KeyboardInterrupt: 

qxu5pjgrr7gq02jrvo1zeicdncqpln
